In [ ]:
!nvcc --version

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc4jupyter

**Question1 (Q1)**

In [47]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(float *arr, int size,float *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ float partialSum[];

unsigned int thread_id = blockIdx.x * blockDim.x + threadIdx.x ;

unsigned int t = threadIdx.x;

partialSum[t] = arr[thread_id];

if(thread_id >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 float* array = nullptr;  // Pointer to the array
 float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     float *array_device;
     float *result_device;

     result = (float*)malloc(sizeof(float));
     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"final sum is: "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q1_2_31_2_33.cu


In [48]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1
!./Q1 "input1000.txt"

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

**Question2 (Q2)**

In [26]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>

using namespace std;

#define block_dim 1024
#define search_interval 16
#define shared_size block_dim*search_interval

// Kernel definition
__global__ void BinarySearch(float *array, int size, float target, float *index, bool* found)
{

    *index = -1;

    if(*found == false) { // not found yet

        int thread_id = threadIdx.x;
        __shared__ float shared_array[shared_size];


        for(int i = 0; i < search_interval; i++) {
            int shared_location = thread_id * search_interval + i;
          //  int location = shared_location + shared_size * blockIdx.x;
          int location = shared_location;


            if(location < size) {
              shared_array[shared_location] = array[location];
            }
            else {
              shared_array[shared_location] = INT_MAX;
            }
        }
        __syncthreads();

        int start = 0;
        int end = search_interval - 1;
        int mid = (start + end) / 2;

        float *thread_part = &shared_array[thread_id * search_interval];

        while(start <= end && *found == false)
        {
            if(thread_part[mid] < target) {
                start = mid + 1;
            }
            else if(thread_part[mid] > target) {
                end = mid - 1;
            }
            else {
                *found = true;
               // *index = mid + thread_id * search_interval + shared_size * blockIdx.x;
               *index = mid + thread_id * search_interval ;

            }

            mid = (start + end) / 2;
        }

    }
}


int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    float* array = nullptr;  // Pointer to the array
    float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }


      bool * flag;
    // host to device allocation
     float *array_device;
     float *result_device;
      bool * flag_device;

     result = (float*)malloc(sizeof(float));
     flag = (bool*)malloc(sizeof(bool));
     *flag = false;

     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));
     cudaMalloc((void**)&flag_device,sizeof(bool));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(flag_device, flag, sizeof(bool), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1,1);
    BinarySearch<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count, target,result_device, flag_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"target index : "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q2_2_31_2_33.cu


In [28]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2
!./Q2 "input1000.txt" 552


Compilation terminated.
nvcc error   : 'cudafe++' died due to signal 2 
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
2

In [29]:
!nvprof ./Q2 "input1000.txt" 1222

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27